In [54]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2
from scipy import ndimage
from matplotlib import pyplot
import scipy
from PIL import Image
from skimage import measure
import math
from tqdm import tqdm_notebook as tqdm
import h5py
from imageio import imread
import pickle
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/xupeng/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['imread', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# main function

In [117]:
def postsynapticArea(axonVolume,muscleVolume,miplevel,axonId,muscleId,dilationXY,dilationZ):
    '''
    calculate the postsynaptic area of a given axon at a given NMJ
    
    input:
        axonVolume: [3D array] the original segmentation image volume of axon.
        muscleVolume: [3D array] the original segmentation image volume of muscle.
        mipLevel: [int] the mipLevel of original segmentation image.
        axonId: [int] the Id of desired axon.
        muscleId: [int] the Id of desired muscle.
        dilationXY: [int] dilation distance on XY plane, in nm
        dilationZ: [int] dilation distance on Z axis, in nm
    output:
        area: [int] the postsynaptic area of the given axon at the given NMJ
    '''
    print ('preprocessing ............')
    axon,muscle = preprocessing(axonVolume,muscleVolume,miplevel,axonId,muscleId)
    print ('dilating ............')
    daxon = dilation(axon,dilationXY/4,dilationZ/4)
    overlap = (daxon == 1) * (muscle == 1)
    print ('calculating overlap ............')
    area = boundryArea(overlap,daxon)
    
    return area,axon,muscle,daxon

# functions called by the main function

### interpolation

In [26]:
def preprocessing(axonVolume,muscleVolume,miplevel,axonId,muscleId):
    
    #preserve the desired axon and muscle
    axon = (axonVolume == axonId).astype('uint8')
    muscle = (muscleVolume == muscleId).astype('uint8')
    
    #bounding box according to muscle
    zMIN,zMAX,xMIN,xMAX,yMIN,yMAX = boundingBox(muscle)
    zMIN_ = max(0,zMIN-int(50/int((16/(2**miplevel)))))
    zMAX_ = min(axon.shape[0]-1,zMAX+int(50/int((16/(2**miplevel))))+1)
    xMIN_ = max(0,xMIN-50)
    xMAX_ = min(axon.shape[1]-1,xMAX+50)
    yMIN_ = max(0,yMIN-50)
    yMAX_ = min(axon.shape[2]-1,yMAX+50)
    axonB = axon[zMIN_:zMAX_+1,xMIN_:xMAX_+1,yMIN_:yMAX_+1]
    muscleB = muscle[zMIN_:zMAX_+1,xMIN_:xMAX_+1,yMIN_:yMAX_+1]
    
    #initiate the interpolated volume
    z = int(axonB.shape[0] * (16/(2**miplevel)))
    x = axonB.shape[1] 
    y = axonB.shape[2] 
    interpolatedAxon = np.zeros((z,x,y),dtype = 'uint8')
    interpolatedMuscle = np.zeros((z,x,y),dtype = 'uint8')
    
    #interpolation
    for i in range(y):
        res = cv2.resize(axonB[:,:,i],(x,z),interpolation = cv2.INTER_LINEAR)
        interpolatedAxon[:,:,i]=res
    for i in range(y):
        res = cv2.resize(muscleB[:,:,i],(x,z),interpolation = cv2.INTER_LINEAR)
        interpolatedMuscle[:,:,i]=res
        
    return interpolatedAxon,interpolatedMuscle

### dilation

In [27]:
def dilation(imgVolume,dilationXY,dilationZ):
    dilatedVolume = np.zeros(imgVolume.shape,dtype = "bool")
    for i in tqdm(range(imgVolume.shape[0])):
        
        #dilate in XY plane
        dt = ndimage.distance_transform_edt(1-imgVolume[i]) 
        dilatedSection = (dt<dilationXY)
        dilatedVolume[i] += dilatedSection
        
        #dilate in Z axis
        for j in range(int(dilationZ))[1:]:
            if i-j >= 0:
                dilatedVolume[i-j] += dilatedSection
            if i+j < imgVolume.shape[0]:
                dilatedVolume[i+j] += dilatedSection
                
    return dilatedVolume

### boundry

In [28]:
def boundryArea(overlap,daxon):
    zMIN,zMAX,xMIN,xMAX,yMIN,yMAX = boundingBox(overlap)
    dt_daxon = ndimage.distance_transform_edt(daxon[zMIN:zMAX+1,xMIN:xMAX+1,yMIN:yMAX+1]) 
    false_boundry = (dt_daxon == 1)
    dt_ol = ndimage.distance_transform_edt(overlap[zMIN:zMAX+1,xMIN:xMAX+1,yMIN:yMAX+1]) 
    true_boundry = (dt_ol == 1)
    inner = true_boundry * false_boundry
    boundry = true_boundry ^ inner
    boundryArea = np.sum(boundry)
    return boundryArea

### bounding box

In [29]:
def boundingBox(imgVolume):
    zMIN = 0
    zMAX = imgVolume.shape[0]-1
    for i in range(imgVolume.shape[0]):
        sum = np.sum(imgVolume[i,:,:])
        if zMIN == 0 and sum > 0:
            zMIN = i
        if sum > 0:
            zMAX = i
            
    xMIN = 0
    xMAX = imgVolume.shape[1]-1
    for i in range(imgVolume.shape[1]):
        sum = np.sum(imgVolume[:,i,:])
        if xMIN == 0 and sum > 0:
            xMIN = i
        if sum > 0:
            xMAX = i
    
    yMIN = 0
    yMAX = imgVolume.shape[2]-1
    for i in range(imgVolume.shape[2]):
        sum = np.sum(imgVolume[:,:,i])
        if yMIN == 0 and sum > 0:
            yMIN = i
        if sum > 0:
            yMAX = i
            
    return zMIN,zMAX,xMIN,xMAX,yMIN,yMAX